In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.1MB/s 
     |████████████████████████████████| 2.9MB 43.5MB/s 
     |████████████████████████████████| 1.1MB 54.3MB/s 
     |████████████████████████████████| 890kB 41.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5559e71aac57c112954dc0eb015791dbb7f118128dfb7c7282e9b97c2a427e6f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
ROOT_PATH = "/content/drive/My Drive/Magíster 2020"

In [ ]:
puc_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puc_train_70.json", orient="records")
puc_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puc_test_30.json", orient="records")

puj_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puj_train_70.json", orient="records")
puj_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/puj_test_30.json", orient="records")

aurora_train = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_train_70.json", orient="records")
aurora_test = pd.read_json(f"{ROOT_PATH}/Ordenado/new_train_test_data/aurora_test_30.json", orient="records")

train = pd.concat([puc_train, puj_train, aurora_train], ignore_index=True)
test = pd.concat([puc_test, puj_test, aurora_test], ignore_index=True)

In [ ]:
MAX_LENGTH = 512

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data: pd.DataFrame = data
        self.abstracts: np.ndarray = data["clean_abstract"].values
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=False)
        self.titles: np.ndarray = data["title"].values
        self.labels: np.ndarray = data["first_sdg"].values

    def __len__(self):
        return len(self.abstracts)

    def __getitem__(self, index):
        abstract_text = str(self.abstracts[index])
        abstract_text = " ".join(abstract_text.split())

        title_text = str(self.titles[index])
        title_text = " ".join(title_text.split())

        abstract_inputs = self.tokenizer.encode_plus(
            abstract_text,
            None,
            add_special_tokens=True,
            max_length=MAX_LENGTH,
            return_token_type_ids=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        abstract_ids = abstract_inputs['input_ids']
        abstract_mask = abstract_inputs['attention_mask']
        abstract_token_type_ids = abstract_inputs["token_type_ids"]

        title_inputs = self.tokenizer.encode_plus(
            title_text,
            None,
            add_special_tokens=True,
            max_length=MAX_LENGTH,
            return_token_type_ids=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        title_ids = title_inputs['input_ids']
        title_mask = title_inputs['attention_mask']
        title_token_type_ids = title_inputs["token_type_ids"]

        return {
            'abstract_ids': abstract_ids,
            'abstract_mask': abstract_mask,
            'abstract_token_type_ids': abstract_token_type_ids,
            'title_ids': title_ids,
            'title_mask': title_mask,
            'title_token_type_ids': title_token_type_ids,
            'label': torch.tensor(self.labels[index], dtype=torch.long)
        }

In [ ]:
train_dataset = MyDataset(train)
test_dataset = MyDataset(test)

In [ ]:
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(train_dataset, **train_params)
testing_loader = DataLoader(test_dataset, **test_params)

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained("roberta-base", output_attentions=True, return_dict=True)
        self.pre_classifier = torch.nn.Linear(768 * 2, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 18)
    
    def forward(self, batch):
        abstract_ids = torch.reshape(batch["abstract_ids"].to(device, dtype = torch.long), (TRAIN_BATCH_SIZE,MAX_LENGTH))
        abstract_mask = torch.reshape(batch["abstract_mask"].to(device, dtype = torch.long), (TRAIN_BATCH_SIZE,MAX_LENGTH))
        abstract_token_type_ids = torch.reshape(batch["abstract_token_type_ids"].to(device, dtype = torch.long), (TRAIN_BATCH_SIZE,MAX_LENGTH))
        title_ids = torch.reshape(batch["title_ids"].to(device, dtype = torch.long), (TRAIN_BATCH_SIZE,MAX_LENGTH))
        title_mask = torch.reshape(batch["title_mask"].to(device, dtype = torch.long), (TRAIN_BATCH_SIZE,MAX_LENGTH))
        title_token_type_ids = torch.reshape(batch["title_token_type_ids"].to(device, dtype = torch.long), (TRAIN_BATCH_SIZE,MAX_LENGTH))
        X1 = self.roberta(input_ids=abstract_ids, attention_mask=abstract_mask, token_type_ids=abstract_token_type_ids)
        abstract_attentions = X1.attentions
        X1_pooler = X1.pooler_output
        X2 = self.roberta(input_ids=title_ids, attention_mask=title_mask, token_type_ids=title_token_type_ids)
        title_attentions = X2.attentions
        X2_pooler = X2.pooler_output
        X = torch.cat([X1_pooler, X2_pooler], dim=1)
        output = self.pre_classifier(X)
        output = torch.nn.ReLU()(output)
        output = self.dropout(output)
        output = self.linear(output)
        return abstract_attentions, title_attentions, output

In [ ]:
model = Classifier()
model = model.to(device)

In [ ]:
LEARNING_RATE = 1e-05
EPOCHS = 3
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
loss_list=[]
model.train()
for epoch in range(EPOCHS):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    for index, batch in tqdm(enumerate(training_loader, 0)):
        label = batch["label"].to(device, dtype = torch.long)
        optimizer.zero_grad()
        _, _, output = model(batch)
        loss = criterion(output, label)
        tr_loss += loss.item()
        prob = torch.nn.functional.softmax(output, dim=1)
        big_val, big_idx = prob.topk(1, dim=1)
        n_correct += calcuate_accuracy(big_idx, label)
        nb_tr_steps += 1
        nb_tr_examples+=label.size(0)
        if index % 500 == 0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 500 steps: {loss_step}")
            print(f"Training Accuracy per 500 steps: {accu_step}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.data)
    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

1it [00:00,  2.93it/s]

Training Loss per 500 steps: 2.7877635955810547
Training Accuracy per 500 steps: 100.0


502it [00:57,  8.04it/s]

Training Loss per 500 steps: 2.3376013446234896
Training Accuracy per 500 steps: 35.12974051896207


1002it [01:54,  8.58it/s]

...

Training Loss per 500 steps: 0.7588527450671448
Training Accuracy per 500 steps: 77.44945567651634


5002it [09:35,  8.69it/s]

Training Loss per 500 steps: 0.7651990653764587
Training Accuracy per 500 steps: 77.18456308738253


5489it [10:31,  8.70it/s]

The Total Accuracy for Epoch 2: 77.20896338130807
Training Loss Epoch: 0.7711411210868914
Training Accuracy Epoch: 77.20896338130807


In [ ]:
torch.save(model, f"{ROOT_PATH}/Ordenado/with_titles/2020-11-09_roberta_original_with_titles.pt")

In [ ]:
model.eval()
tr_loss = 0
n_correct = 0
nb_tr_steps = 0
nb_tr_examples = 0
y_pred = []
y_true = []
y_all_probs = []
with torch.no_grad():
    for batch in testing_loader:
        label = batch["label"].to(device, dtype = torch.long)
        y_true.append(label.cpu().numpy()[0])
        optimizer.zero_grad()
        _, _, output = model(batch)
        loss = criterion(output, label)
        tr_loss += loss.item()
        prob = torch.nn.functional.softmax(output, dim=1)
        _, big_idx = prob.topk(1, dim=1)
        _, all_ordered = prob.topk(18, dim=1)
        y_pred.append(big_idx.cpu().numpy()[0][0])
        y_all_probs.append(all_ordered[0][:-1].cpu().numpy().tolist())

In [ ]:
y_true == test["first_sdg"].tolist()

True

In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        13
           2       0.62      0.45      0.52        71
           3       0.90      0.89      0.90       771
           4       0.80      0.87      0.83       201
           5       0.73      0.80      0.77       122
           6       0.55      0.79      0.65        91
           7       0.58      0.76      0.66       109
           8       0.50      0.25      0.33        32
           9       0.64      0.32      0.42       136
          10       0.83      0.62      0.71        79
          11       0.70      0.73      0.71       137
          12       0.35      0.38      0.37       106
          13       0.58      0.50      0.54       116
          14       0.58      0.81      0.68        59
          15       0.63      0.66      0.65       149
          16       0.66      0.75      0.70       109
          17       0.34      0.27      0.30        45

    accuracy              

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_true = np.asarray(y_true)
y_pred = np.asarray(y_pred)

misclassified = np.where(y_true != y_pred)[0]

In [ ]:
len(misclassified)

657

In [ ]:
print(test.iloc[misclassified[0]]["title"])
print(test.iloc[misclassified[0]]["clean_abstract"])
print(f"True label: {test.iloc[misclassified[0]]['first_sdg']}")
print(f"Predicted label: {y_pred[misclassified[0]]}")
print(f"Second most probable label: {y_all_probs[misclassified[0]][1]}")

Fgf10 And Sox9 Are Essential For The
Establishment Of Distal Progenitor Cells During Mouse Salivary Gland Development
Salivary glands are formed by branching morphogenesis with epithelial progenitors forming a network of ducts and acini (secretory cells). During this process, epithelial progenitors specialise into distal (tips of the gland) and proximal (the stalk region) identities that produce the acini and higher order ducts, respectively. Little is known about the factors that regulate progenitor expansion and specialisation in the different parts of the gland. Here, we show that Sox9 is involved in establishing the identity of the distal compartment before the initiation of branching morphogenesis. Sox9 is expressed throughout the gland at the initiation stage before becoming restricted to the distal epithelium from the bud stage and throughout branching morphogenesis. Deletion of Sox9 in the epithelium results in loss of the distal epithelial progenitors, a reduction in prolifera

In [ ]:
print(test.iloc[misclassified[-1]]["title"])
print(test.iloc[misclassified[0]]["clean_abstract"])
print(f"True label: {test.iloc[misclassified[-1]]['first_sdg']}")
print(f"Predicted label: {y_pred[misclassified[-1]]}")
print(f"Second most probable label: {y_all_probs[misclassified[-1]][1]}")

Program For The Development Of Attitudes Of Equality Of Gender In Classes Of Physical Education In Schools
Salivary glands are formed by branching morphogenesis with epithelial progenitors forming a network of ducts and acini (secretory cells). During this process, epithelial progenitors specialise into distal (tips of the gland) and proximal (the stalk region) identities that produce the acini and higher order ducts, respectively. Little is known about the factors that regulate progenitor expansion and specialisation in the different parts of the gland. Here, we show that Sox9 is involved in establishing the identity of the distal compartment before the initiation of branching morphogenesis. Sox9 is expressed throughout the gland at the initiation stage before becoming restricted to the distal epithelium from the bud stage and throughout branching morphogenesis. Deletion of Sox9 in the epithelium results in loss of the distal epithelial progenitors, a reduction in proliferation and a 

In [ ]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py


def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
test["predicted"] = y_pred
test["all_probs"] = y_all_probs

In [ ]:
actual = [[pred] for pred in test["first_sdg"]]

In [ ]:
predicted = test["all_probs"].tolist()

In [ ]:
for i in range(2, 6):
    print(f"MAP@{i} = {mapk(actual, predicted, i)}")

MAP@2 = 0.7794117647058824
MAP@3 = 0.795751633986928
MAP@4 = 0.8031045751633986
MAP@5 = 0.8085606706450695


In [ ]:
def correct_label_not_in_k_top_predicted(row: pd.Series, k: int) -> bool:
    return row["first_sdg"] in row["all_probs"][:k]

In [ ]:
for i in range(2, 6):
    test[f"sdg_in_{i}"] = test.apply(lambda row: correct_label_not_in_k_top_predicted(row, i), axis=1)
    missing = len(test[test[f"sdg_in_{i}"].eq(False)])
    print(f"{missing}/{len(test)} ({missing / len(test) * 100}%) with correct label not into the first {i} labels predicted")

378/2346 (16.11253196930946%) with correct label not into the first 2 labels predicted
263/2346 (11.210571184995738%) with correct label not into the first 3 labels predicted
194/2346 (8.269394714407502%) with correct label not into the first 4 labels predicted
130/2346 (5.541346973572037%) with correct label not into the first 5 labels predicted


In [ ]:
test.to_json(f"{ROOT_PATH}/Ordenado/with_titles/2020-11-11_roberta_test.json", orient="records", force_ascii=False)